#Reference
https://stackabuse.com/guide-to-parsing-html-with-beautifulsoup-in-python/ <br>
https://medium.com/analytics-vidhya/web-scraping-for-beginners-using-python-73f0966d895f <br>
https://medium.com/@meenuravi786/web-scraping-book-data-using-python-eb2e761fdc14<br>



In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Example1

In [ ]:
SIMPLE_HTML = """ <html> <head></head>
<body>
  <title>
      <h1>This is the Title</h1>
  </title>
  <p class = "subtitle"> Sutitle is here</p>
  <p> Another p without class </p>
    <ul>
      <li> Ant </li>
      <li> Bat </li>
      <li> Cat </li>
      <li> Dog </li>
    <ul>
</body>
</html>"""

print(SIMPLE_HTML)

In [ ]:
soup = BeautifulSoup(SIMPLE_HTML,'html.parser')
print(soup.title)

In [ ]:
print(soup.title.text)
#print(soup.title.string)

#สร้าง function

In [ ]:
def find_title():
  print(soup.find('h1'))
  print(soup.find('h1').string)


find_title()

In [ ]:
def find_title():
  print(soup.find('title'))
  print(soup.find('title').text)

find_title()

In [ ]:
def find_list_item():
  list_item = soup.find_all('li')
  print(list_item)
  content = [e.string for e in list_item]
  print(content)


find_list_item()

In [ ]:
def find_subtitle():
  paragraph = soup.find('p',{'class':'subtitle'})
  print(paragraph.string)

find_subtitle()

In [ ]:
def find_other_paragraph():
  paragraph = soup.find_all('p')
  print(paragraph)

  other_p = [p for p in paragraph if 'subtitle' not in p.attrs.get('class',[])]
  print("ans =",other_p[0].string)

find_other_paragraph()

# More Complex HTML Parsing

In [ ]:
response = requests.get('http://books.toscrape.com')
soup = BeautifulSoup(response.content,'html.parser')
soup

#การใช้ select_one, select
Create function to get book title

In [ ]:
  #show title and href of first book
  #select_one(~) method returns the first tag that match a particular criteria.
locator = 'article.product_pod h3 a'
item = soup.select_one(locator)
item_name = item.attrs['title']
print(item_name)
item_link = item.attrs['href']
print(item_link)

In [ ]:
def get_title_name():
  locator = 'article.product_pod h3 a'
  item = soup.select_one(locator)
  item_name = item.attrs['title']
  print(item_name)


get_title_name()

In [ ]:
def getall_title_name(soup):
    locator = 'article.product_pod h3 a'
    item = soup.select(locator)
    Book_titles = []
    for i in range(len(item)):
      #print(item[i]['title'])
      aTitle= item[i]['title']
      Book_titles.append(aTitle)
    return Book_titles

#findall_title_name(soup)
data_title = getall_title_name(soup)
data_title

Create function to get price using find_all

In [ ]:
Book_price_tags = soup.find_all('p', class_ = 'price_color')
Book_price_tags[0].text

In [ ]:
def get_book_price(soup):
    Book_price_tags = soup.find_all('p', class_ = 'price_color')
    Book_price = []
    for tags in Book_price_tags:
        aPrice = tags.text
        Book_price.append(aPrice)
    return Book_price

get_book_price(soup)

In [ ]:
def getall_book_price(soup):
    Book_price_tags = soup.find_all('p', class_ = 'price_color')
    Book_price = []
    for tags in Book_price_tags:
        aPrice= tags.text
        #print(aPrice)
        Book_price.append(aPrice)
        #print(Book_price)
    return Book_price

data_price = get_book_price(soup)
data_price

Create function to get all rating

In [ ]:
def findall_rating(soup):
  locator = 'article.product_pod p.star-rating'
  star_rating_tag = soup.select(locator)
  #print(star_rating_tag)
  other_p = [p.attrs['class'] for p in star_rating_tag]
  #print(other_p)
  Book_rate = []
  for rate in other_p:
      Book_rate.append(rate[1])
  return Book_rate

data_rate = findall_rating(soup)
data_rate

#Prepare data frame

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['title', 'price','rating'])
#set max_colwidth
#pd.set_option('display.max_colwidth', None)
temp = {}


# Workshop Scraping URL of detial of each book

In [ ]:
result = soup.find_all(class_="product_pod")[0].find("a").attrs["href"]
result

In [ ]:
url_book = soup.find_all(class_="product_pod")[0].find("a").attrs["href"]
print(url_book)

In [ ]:
base_url = "http://books.toscrape.com/"
new_url = base_url + url_book
print(new_url)

In [ ]:
# list of all books on the overview page
books = soup.find_all(class_="product_pod")
base_url = "http://books.toscrape.com/"
new_url = []

for book in books:
    book_url = book.find("a").attrs["href"]
    real_url = base_url + book_url
    #print(new_url)
    new_url.append(real_url)

# print the first five urls
#print(new_url[0:5])
new_url

In [ ]:
# Initialize the DataFrame
df = pd.DataFrame(columns=['title', 'price', 'rating', 'url'])

# Create a list to collect rows
rows = []
for i in range(len(data_title)):
    temp = {
        'title': data_title[i],
        'price': data_price[i],
        'rating': data_rate[i],
        'url': new_url[i]
    }
    rows.append(temp)

# Convert list of rows into a DataFrame
df = pd.DataFrame(rows)

#Write dataframe to CSV or Excel

In [ ]:
df.to_csv('Bookdata.csv', index=True, encoding = 'utf-8')
#df.to_csv('ฺBookdata.csv', index=False)
#df.to_excel('ฺBookdata.xls', index=True)